In [ ]:
import pandas as pd
from assetAllocation import AssetAllocation
import commonHelper
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_stock import DB_Stock
from db_nyse import DB_NYSE
from commonHelper import EDateType, EFinancialStatementType
from datetime import datetime
import yfinance as yf
from portfolio import Portfolio
import numpy as np
from scipy.stats import norm
from dateutil.parser import parse
import itertools



df = pd.read_csv('2025-Q1_rank.csv')
df = df.drop(columns=['Unnamed: 0'])

symbols = df['Symbol'].to_list()[:10]
start_date = "2025-05-30"
end_date = "2025-07-18"

df = AssetAllocation.get_stock_data_with_ma(symbols=symbols, start_date=start_date, end_date=end_date, mas=[10], type='ma_month')
df = AssetAllocation.filter_close_last_month(df)
df = AssetAllocation.strategy_evenly(df, ratios=[10,10,10,10,10,10,10,10,10,10], interval=12)
display(df)


Portfolio.show_portfolio()

,Date,Symbol,Result Close,MMA_10,End Balance,Restart Balance,Balance,Can Rebalance
0,2025-05-30,"[OPFI, DDI, EZPW, GNE, CWCO, PRDO, UGI, CALM, ...","[12.67, 9.96, 13.45, 21.14, 27.03, 34.17, 36.0...","[8.53, 11.81, 13.03, 15.92, 25.76, 26.01, 29.9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000....",10000,False
1,2025-06-30,"[OPFI, DDI, EZPW, GNE, CWCO, PRDO, UGI, CALM, ...","[13.99, 9.57, 13.88, 26.88, 30.02, 32.69, 36.4...","[9.45, 11.31, 13.19, 16.93, 25.98, 27.02, 31.0...","[1104.18, 960.84, 1031.97, 1271.52, 1110.62, 9...","[1104.18, 960.84, 1031.97, 1271.52, 1110.62, 9...",10903,False
2,2025-07-17,"[OPFI, DDI, EZPW, GNE, CWCO, PRDO, UGI, CALM, ...","[11.65, 9.94, 14.23, 20.33, 29.29, 29.71, 36.0...","[10.15, 10.66, 13.49, 17.34, 26.39, 27.77, 32....","[919.49, 997.99, 1057.99, 961.68, 1083.61, 869...","[919.49, 997.99, 1057.99, 961.68, 1083.61, 869...",10177,False
